# UNR - FCEIA 
## Tecnicatura Universitaria en Programación 
### NLP: Trabajo Práctico N°1 

---

**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : C-7438/1

# Importamos librerías necesarias

In [26]:
import os
import pandas as pd
import pickle
from typing import List, Dict, Any
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Carga de datasets

In [7]:
JUEGOS_PATH: str = os.path.join(os.getcwd(), 'data', 'bgg_database.csv')
PELICULAS_PATH: str = os.path.join(os.getcwd(), 'data', 'IMDB-Movie-Data.csv')
LIBROS_PATH: str = os.path.join(os.getcwd(), 'data', 'dataset_libros.csv')

In [8]:
dataset_juegos: pd.DataFrame = pd.read_csv(JUEGOS_PATH)
dataset_peliculas: pd.DataFrame = pd.read_csv(PELICULAS_PATH)
dataset_libros: pd.DataFrame = pd.read_csv(LIBROS_PATH)

# Modelo de Análisis de Sentimientos

Creo un dataset sencillo para entrenar al clasificador.

In [9]:
ESTADOS_ANIMO_PATH: str = os.path.join(os.getcwd(), 'data', 'estados_de_animo.csv')

In [10]:
df_estados_de_animo: pd.DataFrame = pd.read_csv(ESTADOS_ANIMO_PATH)

X: pd.Series = df_estados_de_animo['prompt']
y: pd.Series = df_estados_de_animo['estado_animo']

Hacemos un split de los datos y creamos un pipeline de trabajo utilizando el clasificador **MultinomialNB**.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo_animo = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [12]:
modelo_animo.fit(X_train, y_train)

y_pred = modelo_animo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Alegre       1.00      0.96      0.98        79
 Melancólico       1.00      1.00      1.00        65
 Ni fu ni fa       0.96      1.00      0.98        76

    accuracy                           0.99       220
   macro avg       0.99      0.99      0.99       220
weighted avg       0.99      0.99      0.99       220



Creamos una función para clasificar el prompt del usuario.

In [13]:
def clasificar_animo(prompt_usuario: str) -> str:
    estado_animo_predicho = modelo_animo.predict([prompt_usuario])[0]
    
    return estado_animo_predicho

In [10]:
nuevo_prompt = "La vida no significa nada"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Ni fu ni fa


In [14]:
nuevo_prompt = "Me siento feliz"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Alegre


In [15]:
nuevo_prompt = "El vacio se siente en mi"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Melancólico


Exporto el modelo

In [13]:
# ESTADO_ANIMO_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_estado_animo.pickle')

# pickle.dump(modelo_animo, open(ESTADO_ANIMO_MODEL_PATH, 'wb'))

# Análisis Datasets

## Análisis Dataset Juegos
Como Vemos en el dataset de juegos no se presentan datos nulos.

In [16]:
dataset_juegos.isna().sum()

rank                0
game_name           0
game_href           0
geek_rating         0
avg_rating          0
num_voters          0
description         0
yearpublished       0
minplayers          0
maxplayers          0
minplaytime         0
maxplaytime         0
minage              0
avgweight           0
best_num_players    0
designers           0
mechanics           0
categories          0
dtype: int64

## Análisis Dataset Películas

In [17]:
dataset_peliculas.isna().sum()

Rank                  0
Title                 0
Genre                 0
Description           0
Director              0
Actors                0
Year                  0
Runtime (Minutes)     0
Rating                0
Votes                 0
Revenue (Millions)    0
Metascore             0
dtype: int64

## Dataset Libros
De los 5984 libros dentro del dataset, solo 53 tiene titulo secundario y solo 2796 libros tienen un autor registrado.

In [18]:
dataset_libros.shape

(5984, 4)

In [19]:
dataset_libros.isna().sum()

Titulo Principal        0
Titulo Secundario    5931
Autor                3188
N° Ref                  0
dtype: int64

# Apartado 1

## Implementación modelo KNN con encoder de tensorflow

In [21]:
# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [25]:
embeding_juegos= embed(dataset_juegos['description']).numpy()
embeding_libros = embed(dataset_libros['Titulo Principal']).numpy()
embeding_peliculas = embed(dataset_peliculas['Description']).numpy()

In [27]:
embedings_totales = np.concatenate([embeding_juegos,embeding_libros, embeding_peliculas ])

In [28]:
n_neighbors = 2 
knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute')
knn.fit(embedings_totales)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=2)

In [29]:
consulta = 'Historia de amor en la playa'
consulta = embed(consulta).numpy()

In [31]:
# Realizamos la búsqueda de los vecinos más cercanos
distances, indices = knn.kneighbors(consulta)

# Imprimimos los vecinos más cercanos y sus distancias
print(f"Consulta: {consulta}")
for j in range(n_neighbors):
    print(f"Vecino {j + 1}: {embedings_totales[indices[i][j]]} - Distancia: {distances[i][j]:.4f}")
print("-" * 40)

Consulta: [[ 1.05126444e-02  2.67677568e-03  9.77139175e-03  5.74320816e-02
  -2.52461750e-02 -3.52786928e-02  1.34940380e-02 -5.84716089e-02
  -6.25243112e-02  4.54057716e-02  7.82534480e-02  8.18276778e-02
  -4.24334966e-02  9.20067132e-02  1.22572377e-03 -6.46360219e-02
   1.54562853e-02 -1.85786635e-02  3.88166122e-02 -7.14329928e-02
  -5.63296257e-03 -7.20195053e-03 -3.20562162e-02  1.48534132e-02
  -5.17080314e-02  9.38060228e-03 -7.35931052e-03  6.68460503e-02
   3.07073034e-02  2.60826740e-02  1.32062370e-02  6.46084771e-02
   6.97033033e-02 -1.14155225e-02  2.51888819e-02  2.04776451e-02
   4.23050113e-03  9.85113438e-03  9.30242017e-02 -6.85811788e-02
  -2.20581088e-02 -9.87384748e-03  2.04637833e-02  1.22530079e-02
   1.27155455e-02  1.83917712e-02 -2.07506306e-02 -7.90742505e-03
   8.98037478e-03 -4.37534340e-02  4.51561697e-02 -5.42256199e-02
   1.07520157e-02  2.01585311e-02 -2.99148522e-02 -5.18651269e-02
  -3.59747410e-02 -2.15796311e-03  9.47919935e-02  3.18975970e-02


NameError: name 'i' is not defined